https://paperswithcode.com/dataset/oxford-102-flower

Скачаем и разархивируем датасет в папку dataset

In [11]:
!wget 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz' -P dataset

--2022-12-19 15:46:35--  https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.221.128, 64.233.161.128, 173.194.222.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.221.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228813984 (218M) [application/x-compressed-tar]
Saving to: ‘dataset/flower_photos.tgz’

flower_photos.tgz   100%[===================>] 218.21M  9.62MB/s    in 22s     

2022-12-19 15:46:57 (9.78 MB/s) - ‘dataset/flower_photos.tgz’ saved [228813984/228813984]



In [13]:
!cd dataset && tar -xf flower_photos.tgz

In [2]:
import tensorflow as tf

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib

In [3]:
tf_flowers_labels = ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [4]:
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)

NameError: name 'hub' is not defined